In [1]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import least_squares
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import SIR_utils as sir

In [4]:
mandate = pd.read_csv('stay_at_home_and_masks.csv')
def get_date(dataframe, state):
    state = dataframe.loc[dataframe['state'] == state]
    date = np.array(state['mask_date'])[0].split('-')[::-1]
    return date[0] + '-' + date[1] + '-' + date[2]

print(mandate)

   state   mask_date  mask_in_effect mask_end_date stay_at_home_start  \
0     AL  16-07-2021           False    09-04-2021         04-04-2020   
1     AK         NaN           False           NaN         28-03-2020   
2     AZ         NaN           False           NaN         31-03-2020   
3     AR  20-07-2020           False    25-03-2021                NaN   
4     CA  18-06-2020            True           NaN         19-03-2020   
5     CO  17-07-2021            True    03-05-2021         26-03-2020   
6     CT  14-08-2020            True    19-05-2021         23-03-2020   
7     DE  05-01-2021            True           NaN         24-03-2020   
8     DC  22-07-2020            True           NaN         01-04-2020   
9     FL         NaN           False           NaN         03-04-2020   
10    GA         NaN           False           NaN         03-04-2020   
11    HI  17-04-2020            True           NaN         25-03-2020   
12    ID         NaN           False           NaN 

In [21]:
state = "new-jersey"
state_abbr = "NJ"

time_dataframe = pd.read_csv("{}-history.csv".format(state))
#print(time_dataframe)

dates = time_dataframe["date"][::-1]
removed = time_dataframe["positiveCasesViral"][::-1]
indices = np.array(1 - np.isnan(removed)).astype(np.bool)

dates = np.array(dates)[indices]
rem = np.array(removed[indices])

print(dates)
print(rem)

['2020-03-05' '2020-03-06' '2020-03-07' '2020-03-08' '2020-03-09'
 '2020-03-10' '2020-03-11' '2020-03-12' '2020-03-13' '2020-03-14'
 '2020-03-15' '2020-03-16' '2020-03-17' '2020-03-18' '2020-03-19'
 '2020-03-20' '2020-03-21' '2020-03-22' '2020-03-23' '2020-03-24'
 '2020-03-25' '2020-03-26' '2020-03-27' '2020-03-28' '2020-03-29'
 '2020-03-30' '2020-03-31' '2020-04-01' '2020-04-02' '2020-04-03'
 '2020-04-04' '2020-04-05' '2020-04-06' '2020-04-07' '2020-04-08'
 '2020-04-09' '2020-04-10' '2020-04-11' '2020-04-12' '2020-04-13'
 '2020-04-14' '2020-04-15' '2020-04-16' '2020-04-17' '2020-04-18'
 '2020-04-19' '2020-04-20' '2020-04-21' '2020-04-22' '2020-04-23'
 '2020-04-24' '2020-04-25' '2020-04-26' '2020-04-27' '2020-04-28'
 '2020-04-29' '2020-04-30' '2020-05-01' '2020-05-02' '2020-05-03'
 '2020-05-04' '2020-05-05' '2020-05-06' '2020-05-07' '2020-05-08'
 '2020-05-09' '2020-05-10' '2020-05-11' '2020-05-12' '2020-05-13'
 '2020-05-14' '2020-05-15' '2020-05-16' '2020-05-17' '2020-05-18'
 '2020-05-

In [27]:
window = 14
infectious, infectious_rate = sir.compute_number_infectious(rem, window)

rem = rem[:len(infectious)]

population = 8e7
sus = population - infectious - rem
print(sus)

328
[79996422.33855686 79995679.81013119 79994814.00072886 79993807.8436589
 79992647.29118076 79991322.71392128 79989826.89358601 79988152.75838192
 79986293.5207726  79984255.78389212 79982042.10677843 79979657.84001458
 79977107.38811953 79974408.23870263 79971560.76895043 79968586.91581632
 79965497.52514577 79962317.56523323 79959070.73651604 79955768.4005102
 79952424.20699708 79949049.31924197 79945653.4016035  79942240.27295919
 79938823.6330175  79935411.1202624  79932011.10094751 79928623.95298834
 79925267.56049563 79921935.20991254 79918640.16946064 79915380.13083091
 79912151.40123907 79908960.73360057 79905816.34220117 79902725.42419825
 79899699.65014577 79896752.19861516 79893878.56013119 79891085.48760933
 79888381.29774052 79885766.61989796 79883242.23032069 79880816.0266035
 79878484.43658891 79876250.47048105 79874118.13629737 79872091.3298105
 79870160.33236152 79868325.21902333 79866575.5823615  79864903.31304665
 79863309.23505831 79861790.69970846 79860339.92966

In [29]:
times = np.arange(len(rem))
sir_fitting = sir.moving_averages_fits(times, sus, infectious, rem, window=window, 
                                     a_guess=0.01, b_guess=0.01)

In [30]:
print(sir_fitting)

[[5.17138174e-08 2.28569531e-05]
 [8.52956691e-09 4.43951957e-05]
 [4.33176677e-09 4.45547675e-05]
 [2.84706462e-09 3.94483883e-05]
 [1.78459632e-09 1.24506073e-04]
 [1.47098808e-09 1.04603826e-04]
 [1.22750098e-09 9.56700647e-05]
 [1.03941573e-09 9.75306559e-05]
 [8.90273804e-10 1.04985828e-04]
 [7.61887204e-10 1.33050134e-04]
 [6.68316512e-10 1.58205283e-04]
 [5.80179063e-10 1.98060942e-04]
 [5.10771193e-10 2.33755689e-04]
 [4.47064160e-10 2.92140340e-04]
 [3.91093541e-10 3.90531879e-04]
 [3.39142975e-10 5.35736673e-04]
 [2.99568055e-10 6.70752173e-04]
 [2.73556207e-10 8.15909892e-04]
 [2.40835075e-10 1.13026495e-03]
 [2.17084609e-10 1.49016842e-03]
 [1.95404416e-10 1.77448154e-03]
 [1.83333741e-10 2.02870224e-03]
 [1.72861761e-10 2.25084975e-03]]
